<a href="https://colab.research.google.com/github/supertime1/DeepPPG/blob/master/DeepPPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Check if GPU is used

In [0]:
!pip uninstall tensorflow-gpu==2.0.0

In [0]:
!pip install git+https://github.com/analysiscenter/cardio.git

In [0]:
import cardio.batchflow

In [0]:
tf.test.is_built_with_cuda()

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.models import load_model

In [0]:
print(tf.__version__)

In [0]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

## 1.Data Pipeline

### 1.1 Import and Clean

In [0]:
##Need to read data from different subjects (1-12 for training and validation, 13-15 for testing)
def import_clean_data(directory):
    ppg_S = []
    acc_x_S = []
    acc_y_S = []
    acc_z_S = []
    labels=[]

    for subdir, dirs, files in os.walk(directory):
        for file in files:
            with open(file, 'rb') as f:
                data = pickle.load(f, encoding='bytes')          
                #raw acc collected by 32Hz sampling rate
                acc_raw = data[b'signal'][b'wrist'][b'ACC']
                #raw ppg collected by 64Hz sampling rate
                ppg_raw = data[b'signal'][b'wrist'][b'BVP']
                #label for 8s data with 2s shift
                label = data[b'label']

                labels.append(label)

                #convert shape from (len(data),1) to (len(data),None), so that it is compatible with signal.spectrogram
                #ppg
                ppg_raw_re = ppg_raw.reshape(len(ppg_raw))
                #acc
                acc_raw_x = []
                acc_raw_y = []
                acc_raw_z = []
                for i in range(len(acc_raw)):
                    acc_raw_x.append(acc_raw[i][0])
                    acc_raw_y.append(acc_raw[i][1])
                    acc_raw_z.append(acc_raw[i][2])
                acc_x = np.array(acc_raw_x).reshape(len(acc_raw_x))
                acc_y = np.array(acc_raw_y).reshape(len(acc_raw_y))
                acc_z = np.array(acc_raw_z).reshape(len(acc_raw_z))

                #Apply sliding-window (8s with 2s shift) to the dataset, with each segment of 1025 FFT points
                #The result dimension for each channel is 51725 (HR points) x 257 (FFT points:frequency resolution)

                #For ppg use nperseg = 64*8 = 512, overlap= 512 -128 = 384
                ppg_freqs, ppg_times, ppg_Sx = signal.spectrogram(ppg_raw_re, fs=64, window='hanning',
                                                      nperseg=512, noverlap=384,
                                                      detrend=False, scaling='spectrum',nfft=4096)
                #For acc use nperseg = 32*8 = 256, overlap = 256 - 64 = 192
                acc_x_freqs, acc_x_times, acc_x_Sx = signal.spectrogram(acc_x, fs=32, window='hanning',
                                                      nperseg=256, noverlap=192,
                                                      detrend=False, scaling='spectrum',nfft=2048)
                acc_y_freqs, acc_y_times, acc_y_Sx = signal.spectrogram(acc_y, fs=32, window='hanning',
                                                      nperseg=256, noverlap=192,
                                                      detrend=False, scaling='spectrum',nfft=2048)
                acc_z_freqs, acc_z_times, acc_z_Sx = signal.spectrogram(acc_z, fs=32, window='hanning',
                                                      nperseg=256, noverlap=192,
                                                      detrend=False, scaling='spectrum',nfft=2048)

                #Appy frequecy filtering (slice 0-4Hz, i.e.upto 240 bpm) and z-normalisation(zero mean and unit variance)
                ppg_S.append(preprocessing.normalize(ppg_Sx[ppg_freqs <= 4]).T)
                acc_x_S.append(preprocessing.normalize(acc_x_Sx[acc_x_freqs <= 4]).T)
                acc_y_S.append(preprocessing.normalize(acc_y_Sx[acc_y_freqs <= 4]).T)
                acc_z_S.append(preprocessing.normalize(acc_z_Sx[acc_z_freqs <= 4]).T)

    #flat the list to np.array
    ppg_Sx = np.asarray([item for sublist in ppg_S for item in sublist])
    acc_x_Sx = np.asarray([item for sublist in acc_x_S for item in sublist])
    acc_y_Sx = np.asarray([item for sublist in acc_y_S for item in sublist])
    acc_z_Sx = np.asarray([item for sublist in acc_z_S for item in sublist])
    label = np.asarray([item for sublist in labels for item in sublist])
    
    return ppg_Sx,acc_x_Sx,acc_y_Sx,acc_z_Sx,label

In [0]:
os.chdir('C:/Users/57lzhang.US04WW4008/Desktop/Deep Learning/PPG_FieldStudy/training')
rootdir_training = os.getcwd()
ppg_Sx_train,acc_x_Sx_train,acc_y_Sx_train,acc_z_Sx_train,label_train = import_clean_data(rootdir_training)

os.chdir('C:/Users/57lzhang.US04WW4008/Desktop/Deep Learning/PPG_FieldStudy/testing')
rootdir_testing = os.getcwd()
ppg_Sx_test,acc_x_Sx_test,acc_y_Sx_test,acc_z_Sx_test,label_test = import_clean_data(rootdir_testing)

#check dim, all of them should match
print('ppg_Sx_train dim:',ppg_Sx_train.shape)   #(m,257)
print('acc_x_Sx_train dim:',acc_x_Sx_train.shape)#(m,257)
print('acc_y_Sx_train dim:',acc_y_Sx_train.shape)#(m,257)
print('acc_z_Sx_train dim:',acc_z_Sx_train.shape)#(m,257)
print("label_train dim:",label_train.shape)     #(m,)

#check dim, all of them should match
print('ppg_Sx_test dim:',ppg_Sx_test.shape)   #(m,257)
print('acc_x_Sx_test dim:',acc_x_Sx_test.shape)#(m,257)
print('acc_y_Sx_test dim:',acc_y_Sx_test.shape)#(m,257)
print('acc_z_Sx_test dim:',acc_z_Sx_test.shape)#(m,257)
print("label_test dim:",label_test.shape)     #(m,)

### 1.2 Create training and validation sets

In [0]:
#Shuffle the data to break the time sequence
def generate_train_val_data(source,label,train_ratio):
    np.random.seed(1)
    training_length = int(len(source) * train_ratio)
    validation_length = int(len(source) - training_length)
    
    shuffled_set = []
    shuffled_label_set =[]
    randomRows = np.random.randint(len(source), size=len(source))
    for i in randomRows:
        shuffled_set.append(source[i])
        shuffled_label_set.append(label[i])
    train = shuffled_set[0:training_length]
    train_label = shuffled_label_set[0:training_length]
    validation = shuffled_set[:validation_length]
    validation_label = shuffled_label_set[:validation_length]
    return train, train_label, validation, validation_label

In [0]:
#create training and validation, and testing sets
#training/validation set
tv_data_set = np.dstack((ppg_Sx_train,acc_x_Sx_train,acc_y_Sx_train,acc_z_Sx_train)) #convert 2D into 3D
print('input training/validation data dimension is: ', tv_data_set.shape)

#for data_set,split it into train and validation sets for evaluation purpose 
train,train_label,validation,validation_label = generate_train_val_data(tv_data_set,label_train,0.8)
training_set = np.expand_dims(np.array(train),axis=1)
validation_set = np.expand_dims(np.array(validation),axis=1)
training_label_set = np.array(train_label)
validation_label_set = np.array(validation_label)

print("training_set dim:",training_set.shape)
print("training_label_set dim:",training_label_set.shape)
print("validation_set dim:", validation_set.shape)
print("validation_label_set dim:", validation_label_set.shape)

### 1.3 Create testing sets

In [0]:
#testing set
testing_set = np.dstack((ppg_Sx_test,acc_x_Sx_test,acc_y_Sx_test,acc_z_Sx_test)) #convert 2D into 3D
testing_set = np.expand_dims(testing_set,axis=1)
testing_set_label = label_test
print('input test data dimension is: ', testing_set.shape)
print('input test data label dimension is:',testing_set_label.shape)

##2.Tensorflow Convolutional Neural Network (CNN)

### 2.1 Build CNN

In [0]:
# tensorflow input shape = (m,1,257,4)
# Better to experiment with different filter size and strides
## Experiment with batch normalization to see if it improve accuracy
#clear history if necessary
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    #1st Conv2D
    tf.keras.layers.Conv2D(8, (1, 1), strides=(1, 1), 
                           activation='relu', input_shape=(1,257,4)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #2nd Conv2D
    tf.keras.layers.Conv2D(16, (1, 3), strides=(1, 1),
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #3rd Conv2D
    tf.keras.layers.Conv2D(32, (1, 3), strides=(1, 1),
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #4th Conv2D
    tf.keras.layers.Conv2D(64, (1, 3), strides=(1, 1),
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 2),strides=(1, 2)),
    tf.keras.layers.Dropout(0.2),
    #5th Conv2D
    tf.keras.layers.Conv2D(16, (1, 1), strides=(1, 1),
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    #Full connection layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model.summary()

Try learning rate schedule callbacks before increasing training epoches


In [0]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
#train model

#select the best learning rate
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 20))

model.compile(optimizer=Adam(lr=0.001), loss='mse', metrics=['mae'])
` `
#start training
history = model.fit(train_datagen.flow(training_set,training_label_set,batch_size=32),
                    steps_per_epoch=len(training_set)/32,
                    epochs=50,
                    verbose=1,
                    validation_data=validation_datagen.flow(validation_set,validation_label_set,batch_size=32),
                    validation_steps=len(validation_set)/32,
                    callbacks=[lr_schedule]
                   )

Visualize learning rate vs epoches

In [0]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-4, 1e-1,0,1000])

Try the optimized learning rate

In [0]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
model.compile(optimizer=Adam(lr=0.0005), loss='mse', metrics=['mae'])
history = model.fit(train_datagen.flow(training_set,training_label_set,batch_size=32),
                    steps_per_epoch=len(training_set)/32,
                    epochs=200,
                    verbose=1,
                    validation_data=validation_datagen.flow(validation_set,validation_label_set,batch_size=32),
                    validation_steps=len(validation_set)/32,
                   )

### 2.3 Evaluation of the training/validation sets

In [0]:
#model error analysis
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

# Retrieve a list of list results on training and test data sets for each training epoch
mae=history.history['mean_absolute_error']
val_mae=history.history['val_mean_absolute_error']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(mae)) # Get number of epochs

# Plot training and validation mae per epoch

plt.plot(epochs, mae, 'r', label="Training MAE")
plt.plot(epochs, val_mae, 'b', label="Validation MAE")
plt.title('Training and validation MAE')
plt.legend()
plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and validation Loss')
plt.legend()
plt.figure()

Save and load model if necessary

In [0]:
#model.save('my_model1.h5')  # creates a HDF5 file 'my_model.h5' in working directory
#del model  # deletes the existing model
## returns a compiled model
## identical to the previous one
os.chdir('/users/57lzhang/Documents/data/PPG_FieldStudy')
model = tf.keras.models.load_model('my_model_3_.h5')

# 3.Model evaluation

Compare the data predicted by "Deep-PPG" to the one generated by traditional DSP algorithm

In [0]:
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae

In [0]:
os.chdir('/users/57lzhang/Documents/data/PPG_FieldStudy')
model = tf.keras.models.load_model('my_model_1.h5')

In [0]:
def mae_per_activity(directory):
    i=0
    for subdir,dirs,files in os.walk(directory):
        if i<1:
            i+=1
            continue
        else:
            os.chdir(subdir)
            ppg_Sx_test,acc_x_Sx_test,acc_y_Sx_test,acc_z_Sx_test,label_test = import_clean_data(subdir)
            testing_set = np.dstack((ppg_Sx_test,acc_x_Sx_test,acc_y_Sx_test,acc_z_Sx_test)) #convert 2D into 3D
            testing_set = np.expand_dims(testing_set,axis=1)
            hr_estimate = model.predict(testing_set)
            
            #calculate overall MAE
            score = model.evaluate(testing_set, label_test,batch_size=32,verbose=0)
            print('Test subject:',files[0])
            print('Overall MAE:', round(score[1],1))
            
            #plot the figure
            plt.figure(figsize=(15, 6))
            plt.plot(range(len(hr_estimate)), hr_estimate, 'r', label="predicted HR")
            plt.plot(range(len(label_test)), label_test, 'b', label="actual HR")
            plt.legend()
            plt.title(files[0])
            plt.ylabel('Heart Rate(bpm)')
            plt.xlabel('Time(2s)')
            plt.axis([0,5000,40,250])
            
            #calculate MAE per activity
            with open(files[0],'rb') as f:
                data = pickle.load(f, encoding='bytes')          
                #extract the raw 4Hz data
                activity_raw = data[b'activity']            
                time = pd.timedelta_range(0, periods=len(activity_raw), freq='0.25S')     
                df = pd.DataFrame(activity_raw,index = time,columns=list('A'))
                activity = df.resample('2S').max()
                activity=activity[2:-1]
                #Append True and Estimated HR
                activity['HR_True'] = label_test
                activity['HR_estimate'] = hr_estimate
                activity.groupby('A').nunique()

                print('Sitting:',round(mae(activity[activity['A']==1]['HR_True'],activity[activity['A']==1]['HR_estimate']),1))
                print('Climbing:',round(mae(activity[activity['A']==2]['HR_True'],activity[activity['A']==2]['HR_estimate']),1))
                print('Table Soccer:',round(mae(activity[activity['A']==3]['HR_True'],activity[activity['A']==3]['HR_estimate']),1))
                print('Cycling:',round(mae(activity[activity['A']==4]['HR_True'],activity[activity['A']==4]['HR_estimate']),1))
                print('Driving:',round(mae(activity[activity['A']==5]['HR_True'],activity[activity['A']==5]['HR_estimate']),1))
                print('Lunch:',round(mae(activity[activity['A']==6]['HR_True'],activity[activity['A']==6]['HR_estimate']),1))
                print('Walking:',round(mae(activity[activity['A']==7]['HR_True'],activity[activity['A']==7]['HR_estimate']),1))
                print('Working:',round(mae(activity[activity['A']==8]['HR_True'],activity[activity['A']==8]['HR_estimate']),1))
                print('\n')


In [0]:
os.chdir('/users/57lzhang/Documents/data/PPG_FieldStudy/test')
rootdir_testing = os.getcwd()
mae_per_activity(rootdir_testing)